# **Step 03: End-to-End Azure ML Model Deployment Pipeline**

# A. Project Configuration and Environment Setup

In [ ]:
import os
import uuid
from datetime import datetime
from pathlib import Path

# # Define project name and a unique run ID based on current UTC timestamp
PROJECT_NAME        = "leaf-detection"
RUN_ID              = datetime.utcnow().strftime("%Y%m%d%H%M%S")

# Define root directories and important paths
PATH_ROOT           = Path(".").resolve()
PATH_CODE           = PATH_ROOT / "src"                          # Source code directory
PATH_MODEL          = PATH_ROOT / "artifacts" / "model.pth"      # Trained model path
PATH_ENV_FILE       = PATH_ROOT / "environment" / "conda.yaml"   # Conda environment file 

# Azure Machine Learning workspace configuration
AZ_SUBSCRIPTION_ID  = "<subscription_id>"
AZ_RESOURCE_GROUP   = "<resource_group>"
AZ_WORKSPACE_NAME   = "<workspace_name>"

# Deployment configuration for model, endpoint, and environment
CFG_MODEL_NAME      = f"{PROJECT_NAME}-model"                    # Registered model name
CFG_MODEL_VERSION   = f"{RUN_ID}"                                # Model version based on run ID
CFG_ENV_NAME        = f"{PROJECT_NAME}-env"                      # Environment name
CFG_ENV_VERSION     = "1"                                        # Environment version
CFG_ENDPOINT_NAME   = f"{PROJECT_NAME}-ep-{uuid.uuid4().hex[:8]}"  # Unique endpoint name
CFG_DEPLOYMENT_NAME = "blue"                                     # Deployment slot name
CFG_INSTANCE_TYPE   = "Standard_D2as_v4"                         # Compute instance size
CFG_INSTANCE_COUNT  = 1                                          # Number of instances

# B. Initialize Azure ML Client with Default Credentials

In [ ]:
# Read the docs:
# https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python
# https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.mlclient?view=azure-python
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

# Initialize default Azure authentication (uses CLI login, VSCode auth, or managed identity)
credential = DefaultAzureCredential()

# Create MLClient instance to interact with Azure ML workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=AZ_SUBSCRIPTION_ID,
    resource_group_name=AZ_RESOURCE_GROUP,  
    workspace_name=AZ_WORKSPACE_NAME,
)

# Retrieve workspace information
ws = ml_client.workspaces.get(AZ_WORKSPACE_NAME)

print("Resource Group in client:", getattr(getattr(ml_client, "_operation_scope", None), "resource_group_name", None))
print("Connected to workspace:", ws.name, "-", ws.location)

# C. Register the Trained Model in Azure ML

In [ ]:
# Read the docs:
# https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.model?view=azure-python
from azure.ai.ml.entities import Model

# Define model metadata and reference to the local model artifact
AZ_MODEL = Model(
    name=CFG_MODEL_NAME,
    version=CFG_MODEL_VERSION,
    type="custom_model",           # Specifies a generic custom model
    path=str(PATH_MODEL),          # Path to model file or directory
    description=f"Model {PROJECT_NAME} version {CFG_MODEL_VERSION}",
    tags={"project": PROJECT_NAME, "run_id": RUN_ID},
)

# Register (or update if exists) the model in Azure ML workspace
AZ_MODEL = ml_client.models.create_or_update(AZ_MODEL)
print("Registered model ID:", AZ_MODEL.name, "version:", AZ_MODEL.version)

# D. Create and Register the Azure ML Environment

In [ ]:
# create environment folder
os.makedirs("environment", exist_ok=True)

In [ ]:
%%writefile environment/conda.yaml
name: model-deploy-env
channels:
  - pytorch
  - conda-forge
  - defaults
dependencies:
  - python=3.10.11
  - pytorch=2.7.1
  - torchvision=0.22.1
  - numpy=2.1.3
  - scikit-learn=1.7.2
  - matplotlib=3.10.3
  - pandas=2.3.1
  - scipy=1.15.3
  - pip
  - pip:
      - tqdm==4.67.1
      - azureml-inference-server-http
      - azureml-defaults

In [ ]:
# Read the docs:
# https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.environment?view=azure-python
from azure.ai.ml.entities import Environment

# Define environment settings using a Conda file and base Docker image
AZ_ENV = Environment(
    name=CFG_ENV_NAME,
    version=CFG_ENV_VERSION,
    conda_file=str(PATH_ENV_FILE), # Path to Conda environment definition
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

# # Register (or update if exists) environment in Azure ML workspace
AZ_ENV = ml_client.environments.create_or_update(AZ_ENV)
print("Environment created:", AZ_ENV.name, AZ_ENV.version)

# E. Create a Managed Online Endpoint for Real-time Inference

In [ ]:
# Read the docs:
# https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.managedonlineendpoint?view=azure-python
from azure.ai.ml.entities import ManagedOnlineEndpoint

# Define endpoint configuration
EP_ENDPOINT = ManagedOnlineEndpoint(
    name=CFG_ENDPOINT_NAME,
    auth_mode="key",
    description=f"Endpoint for {PROJECT_NAME}",
    tags={"project": PROJECT_NAME}
)

# Create (or update if exists) the endpoint in Azure ML workspace
# Long-running operation
EP_ENDPOINT = ml_client.online_endpoints.begin_create_or_update(EP_ENDPOINT).result()
print("Endpoint created:", EP_ENDPOINT.name)

# F. Create and Configure the Online Deployment for Serving the Model

In [ ]:
# Read the docs:
# https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.managedonlinedeployment?view=azure-python
# https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.codeconfiguration?view=azure-python
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

# Define deployment configuration
EP_DEPLOYMENT = ManagedOnlineDeployment(
    name=CFG_DEPLOYMENT_NAME,                                # Deployment slot name
    endpoint_name=CFG_ENDPOINT_NAME,                         # Target endpoint
    model=f"azureml:{CFG_MODEL_NAME}:{CFG_MODEL_VERSION}",   # Registered model reference
    environment=f"azureml:{AZ_ENV.name}:{AZ_ENV.version}",   # Registered environment reference
    instance_type=CFG_INSTANCE_TYPE,                         # Compute SKU for serving
    instance_count=CFG_INSTANCE_COUNT,                       # Number of instances
    code_configuration=CodeConfiguration(
        code=str(PATH_CODE),                                 # Folder containing scoring script
        scoring_script="scoring.py",                         # Entry script for inference
    ),
)

# Create (or update if exists) the deployment 
# Long-running operation
EP_DEPLOYMENT = ml_client.online_deployments.begin_create_or_update(EP_DEPLOYMENT).result()
print("Deployment ready:", EP_DEPLOYMENT.name)

## Update Traffic and Get Scoring URI

In [ ]:
ep = ml_client.online_endpoints.get(name=CFG_ENDPOINT_NAME)

ep.traffic = {CFG_DEPLOYMENT_NAME: 100}         # Route 100% of traffic to the 'blue' deployment

# Persist the updated endpoint configuration
ep = ml_client.online_endpoints.begin_create_or_update(ep).result() 

SCORING_URI = ep.scoring_uri                    # Get the scoring URI to send requests later
print("Traffic:", ep.traffic)
print("Scoring URI:", SCORING_URI)

## Retrieve Endpoint Keys (Sensitive)

In [ ]:
# Retrieve endpoint keys (sensitive). Keep this separated so you can control execution and visibility
# Note: auth_mode must be "key" for this to return keys
creds = ml_client.online_endpoints.get_keys(name=CFG_ENDPOINT_NAME)

AZML_TOKEN = creds.primary_key                 # Use primary_key for client requests
print("Primary key loaded?", bool(AZML_TOKEN)) # (do not print the actual key)

# G. Test Inference Request to the Azure ML Online Endpoint

In [ ]:
import os, json, base64, requests

# Load a sample image and convert it to Base64 string
image_path = "leaf-disease-dataset/test_downymildew.jpg"
with open(image_path, "rb") as f:
    img_b64 = base64.b64encode(f.read()).decode("utf-8")

# Prepare payload according to scoring.py run() input format
payload = {"image": img_b64}

# Build HTTP headers for authentication and routing
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {AZML_TOKEN}",            # Endpoint key for auth
    "azureml-model-deployment": CFG_DEPLOYMENT_NAME,    # Optional: specify deployment slot
}

# Send POST request for inference
resp = requests.post(SCORING_URI, json=payload, headers=headers, timeout=60)

print("Status:", resp.status_code)
print("Response JSON:", resp.json())